<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 3 Web APIs and NLP

_Authors: Joel Quek (SG)_

# Web Scraping for r/investing

In [96]:
import requests
import pandas as pd
import time
from datetime import datetime

import random

import json
import csv

In [97]:
url = 'https://api.pushshift.io/reddit/search/submission'

# Function to Pull 5000 rows of data

## Current Unix Timestamp

In [98]:
presentDate = datetime.now()
unix_timestamp = datetime.timestamp(presentDate)
print(unix_timestamp)

1667312094.527157


https://www.epochconverter.com/

## Function 

In [99]:
def PushShift5000(sub, size, present):
    url = 'https://api.pushshift.io/reddit/search/submission'
    #-----------------------------------------------------------------
    params ={
        'subreddit': str(sub),
        'size': int(size),
        'before': int(present)
    }
    res = requests.get(url,params)
    data=res.json()
    posts = data['data']
    df=pd.DataFrame(posts)
    max_size = df.shape[0]-1
    #------------------------------------------------------------------
    while df.shape[0] < size:
        params2 ={
            'subreddit': str(sub),
            'size': int(size),
            'before': posts[max_size]['created_utc']
        }
        res2 = requests.get(url,params2)
        data2=res2.json()
        posts = data2['data']
        df2=pd.DataFrame(posts) 
        df=pd.concat([df,df2],ignore_index=True)
    return df

# r/investing

In [100]:
investing = PushShift5000('investing', 3600, unix_timestamp)

In [101]:
investing.head()

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_is_blocked,author_patreon_flair,...,removed_by_category,suggested_sort,author_flair_background_color,author_flair_text_color,banned_by,author_cakeday,link_flair_template_id,link_flair_text,call_to_action,category
0,[],False,hydrocyanide,None,[],None,text,t2_ejq52,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,KyloRenSucks,None,[],None,text,t2_4ydl1bh9,False,False,...,moderator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,Electronic_Ad_701,None,[],None,text,t2_atg5vj2m,False,False,...,moderator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,Quiet-Literature2251,None,[],None,text,t2_9nw9mt1x,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],False,4619,None,[],None,text,t2_3at2tvkk,False,False,...,moderator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
investing.shape[0]

3750

In [103]:
investing.iloc[investing.shape[0]-1]['created_utc']

1662568942

In [108]:
investing2 = PushShift5000('investing', 1250, investing.iloc[investing.shape[0]-1]['created_utc'])

In [112]:
investing2.shape[0]

1498

In [113]:
investing=pd.concat([investing,investing2],ignore_index=True)
investing.shape

(5248, 71)

In [114]:
investing[['subreddit', 'author', 'selftext', 'title']].head(20)

,subreddit,author,selftext,title
0,investing,hydrocyanide,https://www.treasurydirect.gov/savings-bonds/i...,New Series I savings bond rate is 6.89% (0.40%...
1,investing,KyloRenSucks,[removed],Treasury announces new Series I bond rate of 6...
2,investing,Electronic_Ad_701,[removed],Investment question? New to this and 19 I’m eager
3,investing,Quiet-Literature2251,"Hey, this is my current plan. I'm in sales and...",Making substantial money now but not sure what...
4,investing,4619,[removed],What happens in an emerging markets ETF when a...
5,investing,MillerEmma90,[removed],"sFOX Review: Features, Pros, Cons &amp; Best s..."
6,investing,Background_Stock1212,In Morgan Stanley's 2019 The Recession Playboo...,History suggests that the Japanese yen will st...
7,investing,AutoModerator,Have a general question? Want to offer some c...,Daily General Discussion and Advice Thread - N...
8,investing,Darkfuel1,I'm a bit perplexed regarding the difference b...,Question about open int. Vs vol
9,investing,HonestSoldier7,I'm a little curious about this. It's easy eno...,Dogs of the Dow Strategy --- Do you close all ...


# r/stockmarket

In [105]:
stockmarket = PushShift5000('StockMarket', 2500, unix_timestamp)

TypeError: PushShift5000() missing 1 required positional argument: 'present'

In [ ]:
stockmarket.shape

In [ ]:
stockmarket.iloc[2499]['created_utc']

# r/wallstreetbets

In [ ]:
wallstreetbets = PushShift5000('stockmarket', 2500, unix_timestamp)